# Adaptively Sampled MPC

This is a simple demonstration of an adaptively sampled region quadtree used to approximate the solutions to a model-predictive control problem. 

In [80]:
import RegionTrees
using Plots
gr()

Plots.GRBackend()

In [94]:
reload("RegionTrees")

In [241]:
# This module implements our MPC code, which consists of a 1-dimensional
# double-integrator system, forward-euler time-stepping dynamics, and a 
# 10 step prediction window. 

module mpc

using JuMP
using Gurobi
using StaticArrays
using Interpolations
import RegionTrees
import RegionTrees: AbstractRefinery, needs_refinement, get_data, center, vertices

# Solve the MPC problem from a given initial position and velocity
function run_mpc(q0, v0)
    model = Model(solver=GurobiSolver(OutputFlag=0))
    num_time_steps = 10
    dt = 0.1
    u_limit = 3
    C_q = 100
    c_vfinal = 100
    C_u = 1

    @variable model q[1:num_time_steps]
    @variable model v[1:num_time_steps]
    @variable model u[1:num_time_steps]

    @constraint model [i=2:num_time_steps] q[i] == q[i-1] + v[i-1] * dt
    @constraint model [i=2:num_time_steps] v[i] == v[i-1] + u[i-1] * dt
    @constraint model u .<= u_limit
    @constraint model u .>= -u_limit
    @constraint model q[1] == q0
    @constraint model v[1] == v0

    @objective model Min C_q * sum{q[i]^2, i=1:num_time_steps} + c_vfinal * v[end]^2 + C_u * sum{u[i]^2, i=1:num_time_steps}
    solve(model)

    getvalue(q), getvalue(v), getvalue(u)
end

# The MPCRefinery provides enough behavior to implement the 
# RegionTrees AdaptiveSampling interface, which lets us generate
# a quadtree of initial states and their corresponding MPC solutions.
immutable MPCRefinery <: AbstractRefinery
end

function evaluate(cell, point)
    p = (point - cell.boundary.origin) ./ cell.boundary.widths
    cell.data[p[1] + 1, p[2] + 1]
end

# A cell in the quadtree needs refinement if its interpolated solution
# derived from its vertices is not a good fit for the true MPC solution
# at its center. 
function needs_refinement(::MPCRefinery, cell)
    x = center(cell)
    value_interp = evaluate(cell, x)
    value_true = run_mpc(x[1], x[2])[3]
    !isapprox(value_interp, value_true, rtol=1e-1, atol=1e-1)
end

# The data element associated with a cell is a bilinear interpolation
# of the MPC function evaluated at the vertices of the cell.
function get_data(::MPCRefinery, boundary)
    f = v -> run_mpc(v[1], v[2])[3]
    interpolate(f.(vertices(boundary)),
                BSpline(Linear()), OnGrid())
end

# Needed to let us interpolate directly on arrays of control inputs
Base.one{T, N}(::Type{Array{T, N}}) = one(T)

end

mpc

In [243]:
# Simulate a given control function for the double integerator
# model.
function simulate(controller, q0, v0, dt, timespan)
    num_time_steps = timespan / dt
    qs = [q0]
    vs = [v0]
    ts = [0.0]
    q = q0
    v = v0
    for t in 0:dt:(timespan)
        u = controller(t, q, v)
        q += v * dt
        v += u * dt
        push!(qs, q)
        push!(vs, v)
        push!(ts, t)
    end
    ts, qs, vs
end
        

simulate (generic function with 1 method)

In [245]:
controller = (t, q, v) -> begin
    q, v, u = mpc.run_mpc(q, v)
    u[1]
end

(::#49) (generic function with 1 method)

In [246]:
t, q, v = simulate(controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  8.59724e-5,5.36258e-5,2.83979e-5,9.84092e-6,-2.88847e-6,-1.08171e-5,-1.50039e-5,-1.64406e-5,-1.59925e-5,-1.43722e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.000323466,-0.000252279,-0.00018557,-0.000127294,-7.9286e-5,-4.18688e-5,-1.43665e-5,4.48084e-6,1.6203e-5,2.23757e-5])

In [260]:
# State space portrait of the solution, starting from q=10, v=0
plot(q, v, xlim=(-10, 10), ylim=(-10, 10), xlabel="q", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 q 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,184.837 596.063,190.264 595.244,195.691 593.605,201.118 591.147,206.545 587.869,211.972 583.772,217.399 578.856,222.826 573.121,228.253 566.566,233.68 
 559.191,239.107 550.998,244.534 541.985,249.961 532.152,255.388 521.501,260.815 510.029,266.242 497.739,271.669 484.629,277.096 470.7,282.523 455.951,287.95 
 440.383,293.377 423.996,290.931 407.978,288.267 392.362,283.794 377.422,280.519 362.976,276.915 349.074,272.841 335.787,268.148 323.209,262.721 311.45,257.294 
 300.51,251.867 290.39,246.44 281.089,241.013 272.608,235.586 264.946,230.159 258.103,224.732 252.08,219.305 246.876,213.878 242.491,208.451 238.926,203.024 
 236.18,197.597 234.253,192.17 233.146,186.743 232.858,181.316 233.39,175.889 234.741,170.462 236.911,165.035 239.901,159.608 243.71,154.181 248.338,148.754 
 253.786,143.327 260.053,137.9 267.139,132.473 275.045,128.066 283.617,129.582 291.959,133.743 299.673,138.82 306.621,144.118 312.769,149.444 318.112,154.278 
 322.726,159.383 326.569,164.81 329.593,170.237 331.797,175.664 333.182,181.091 333.748,186.209 333.541,191.095 332.596,193.665 331.263,194.994 329.73,195.291 
 328.151,194.788 326.649,193.71 325.309,192.27 324.187,190.652 323.309,189.009 322.679,187.46 322.283,186.09 322.094,185.174 322.043,184.558 322.085,184.186 
 322.184,184.001 322.31,183.952 322.443,183.995 322.571,184.092 322.683,184.217 322.777,184.348 322.85,184.474 322.905,184.584 322.943,184.676 322.968,184.749 
 322.981,184.803 322.986,184.84 322.986,184.864 322.982,184.877 322.976,184.882 322.969,184.881 322.962,184.877 322.956,184.871 322.951,184.865 322.947,184.858 
 322.944,184.852 322.941,184.847 
 "/>

In [259]:
plot(t, q, xlabel="t", ylabel="q", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 t 
 
 
 q 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 40.9296,3.93701 40.9296,3.93701 46.4809,4.75321 52.0322,6.3856 57.5836,8.83419 63.1349,12.099 68.6862,16.18 74.2376,21.0772 79.7889,26.7905 85.3402,33.3201 
 90.8916,40.6659 96.4429,48.8279 101.994,57.8061 107.546,67.6004 113.097,78.211 118.648,89.6378 124.2,101.881 129.751,114.94 135.302,128.815 140.854,143.507 
 146.405,159.015 151.956,175.339 157.508,191.295 163.059,206.85 168.61,221.733 174.162,236.123 179.713,249.971 185.264,263.206 190.816,275.736 196.367,287.449 
 201.918,298.347 207.47,308.428 213.021,317.693 218.572,326.141 224.124,333.774 229.675,340.59 235.226,346.59 240.778,351.774 246.329,356.142 251.88,359.693 
 257.432,362.428 262.983,364.348 268.534,365.45 274.086,365.737 279.637,365.208 285.188,363.862 290.74,361.7 296.291,358.722 301.842,354.928 307.394,350.317 
 312.945,344.89 318.496,338.647 324.048,331.588 329.599,323.713 335.15,315.175 340.702,306.865 346.253,299.18 351.804,292.26 357.356,286.136 362.907,280.813 
 368.458,276.217 374.01,272.389 379.561,269.376 385.112,267.181 390.664,265.801 396.215,265.238 401.766,265.444 407.318,266.385 412.869,267.713 418.42,269.24 
 423.972,270.813 429.523,272.309 435.074,273.644 440.626,274.762 446.177,275.636 451.728,276.264 457.28,276.658 462.831,276.846 468.382,276.897 473.934,276.855 
 479.485,276.757 485.036,276.631 490.588,276.498 496.139,276.372 501.69,276.26 507.242,276.166 512.793,276.093 518.344,276.038 523.896,276 529.447,275.976 
 534.998,275.963 540.55,275.958 546.101,275.958 551.652,275.962 557.204,275.968 562.755,275.975 568.306,275.982 573.858,275.988 579.409,275.993 584.96,275.997 
 590.512,276 596.063,276.002 
 "/>

In [261]:
plot(t, v, xlabel="t", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 t 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 40.9296,128.186 40.9296,140.064 46.4809,151.941 52.0322,163.819 57.5836,175.696 63.1349,187.574 68.6862,199.451 74.2376,211.329 79.7889,223.207 85.3402,235.084 
 90.8916,246.962 96.4429,258.839 101.994,270.717 107.546,282.594 113.097,294.472 118.648,306.349 124.2,318.227 129.751,330.104 135.302,341.982 140.854,353.86 
 146.405,365.737 151.956,360.383 157.508,354.553 163.059,344.763 168.61,337.596 174.162,329.708 179.713,320.792 185.264,310.521 190.816,298.644 196.367,286.766 
 201.918,274.889 207.47,263.011 213.021,251.134 218.572,239.256 224.124,227.379 229.675,215.501 235.226,203.623 240.778,191.746 246.329,179.868 251.88,167.991 
 257.432,156.113 262.983,144.236 268.534,132.358 274.086,120.481 279.637,108.603 285.188,96.7256 290.74,84.848 296.291,72.9705 301.842,61.0929 307.394,49.2154 
 312.945,37.3379 318.496,25.4603 324.048,13.5828 329.599,3.93701 335.15,7.25411 340.702,16.361 346.253,27.4729 351.804,39.0675 357.356,50.7259 362.907,61.3045 
 368.458,72.4767 374.01,84.3542 379.561,96.2318 385.112,108.109 390.664,119.987 396.215,131.189 401.766,141.882 407.318,147.507 412.869,150.415 418.42,151.067 
 423.972,149.964 429.523,147.606 435.074,144.455 440.626,140.913 446.177,137.317 451.728,133.927 457.28,130.929 462.831,128.924 468.382,127.575 473.934,126.761 
 479.485,126.357 485.036,126.25 490.588,126.343 496.139,126.556 501.69,126.829 507.242,127.117 512.793,127.391 518.344,127.633 523.896,127.834 529.447,127.993 
 534.998,128.111 540.55,128.193 546.101,128.245 551.652,128.273 557.204,128.284 562.755,128.283 568.306,128.275 573.858,128.262 579.409,128.247 584.96,128.232 
 590.512,128.219 596.063,128.208 
 "/>

# Approximating the solutions space
Now that we've written down and tested the true MPC solution, we can try to approximate it. The AdaptiveSampling() function will generate a quadtree by iteratively refining each cell in the space until the solutions to the MPC problem within that cell are well-approximated by the interpolation. 

In [251]:
root = RegionTrees.AdaptiveSampling(mpc.MPCRefinery(), SVector(-9, -9), SVector(19, 19))

Cell: RegionTrees.HyperRectangle{2,Float64}([-9.0,-9.0],[19.0,19.0])

In [252]:
# Now we can plot each region in the quadtree. Note that we end up with
# a lot of detail along the switching surface from maximum acceleration
# to maximum deceleration
plt = plot(xlim=(-10, 10), ylim=(-10, 10), legend=nothing, grid=false)
queue = [root]
i = 0
while !isempty(queue) 
    i += 1
    cell = pop!(queue)
    v = hcat(collect(RegionTrees.vertices(cell.boundary))...)
    plot!(v[1,[1,2,4,3,1]], v[2,[1,2,4,3,1]])
    if !RegionTrees.isleaf(cell)
        append!(queue, RegionTrees.children(cell))
    end
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#969600; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 321.005,222.23 325.172,222.23 325.172,219.

# Using the Approximate Solution
We can use the quadtree to produce an approximate controller for our system. Given the current state, we can look up the cell in the quadtree which encompasses that state, then use that cell's interpolation to find our control tape. 

In [253]:
approx_controller = (t, q, v) -> begin
    x = [q, v]
    leaf = RegionTrees.findleaf(root, x)
    u = mpc.evaluate(leaf, x)
    u[1]
end

(::#51) (generic function with 1 method)

In [254]:
t, q, v = simulate(approx_controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  0.0016498,0.00150141,0.00128105,0.00103288,0.000787533,0.000564449,0.000374243,0.000220987,0.000104216,2.05918e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.00148396,-0.00220356,-0.00248174,-0.00245345,-0.00223084,-0.00190206,-0.00153256,-0.00116771,-0.000836245,-0.000553835])

In [258]:
# The results are similar to, but not quite as good as, the 
# values obtained from the exact MPC solution. 
plot(q, v, xlim=(-10, 10), ylim=(-10, 10), xlabel="q", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 q 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,184.837 596.063,190.264 595.244,195.691 593.605,201.118 591.147,206.545 587.869,211.972 583.772,217.399 578.856,222.826 573.121,228.253 566.566,233.68 
 559.191,239.107 550.998,244.534 541.985,249.961 532.152,255.388 521.501,260.815 510.029,266.242 497.739,271.669 484.629,277.096 470.7,282.523 455.951,287.95 
 440.383,293.377 423.996,290.931 407.978,288.267 392.362,283.794 377.422,280.519 362.976,276.915 349.074,272.841 335.787,268.148 323.209,262.721 311.45,257.294 
 300.51,251.867 290.39,246.44 281.089,241.013 272.608,235.586 264.946,230.159 258.103,224.732 252.08,219.305 246.876,213.878 242.491,208.451 238.926,203.024 
 236.18,197.597 234.253,192.17 233.146,186.743 232.858,181.316 233.39,175.889 234.741,170.462 236.911,165.035 239.901,159.608 243.71,154.181 248.338,148.754 
 253.786,143.327 260.053,137.9 267.139,132.473 275.045,128.066 283.617,129.582 291.959,133.743 299.673,138.82 306.621,144.118 312.769,149.444 318.112,154.278 
 322.726,159.383 326.569,164.81 329.593,170.237 331.797,175.664 333.182,181.091 333.748,186.209 333.541,191.095 332.596,193.665 331.263,194.994 329.73,195.291 
 328.151,194.788 326.649,193.71 325.309,192.27 324.187,190.652 323.309,189.009 322.679,187.46 322.283,186.09 322.094,185.174 322.043,184.558 322.085,184.186 
 322.184,184.001 322.31,183.952 322.443,183.995 322.571,184.092 322.683,184.217 322.777,184.348 322.85,184.474 322.905,184.584 322.943,184.676 322.968,184.749 
 322.981,184.803 322.986,184.84 322.986,184.864 322.982,184.877 322.976,184.882 322.969,184.881 322.962,184.877 322.956,184.871 322.951,184.865 322.947,184.858 
 322.944,184.852 322.941,184.847 
 "/>